##Carga de rutas.txt

Las rutas relativas están en un archivo .txt por si hay que realizar modificaciones posteriores.

In [ ]:
rutas = './rutas.txt'

In [ ]:
with open(rutas, 'r') as archivo:
    for linea in archivo.readlines():
        exec(linea.strip(), globals())

##Instalación de librerías


In [ ]:
# !pip install -r requirements

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, hamming_loss
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
from keras.optimizers import Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from random import randint
import time
import sys
import warnings
warnings.filterwarnings('ignore')



##Carga de dataset

In [ ]:
df = pd.read_csv(dataset_entrenamiento_final)

In [ ]:
df.head(2)

,id_orden,id_usuario,perfil_digital,mes,dia,dia_semana,nse,segmento_unico,canal_Autoservicio,canal_BEBIDA,...,producto_78078,producto_78093,producto_78094,producto_78103,producto_78104,producto_78109,producto_78121,producto_78128,producto_78129,producto_78138
0,101-21757542-0,412132,2,6,1,2,1,0,False,False,...,0,0,0,0,0,0,0,0,0,0
1,101-21758309-0,454662,2,6,2,3,0,2,False,False,...,0,0,0,0,0,0,0,0,0,0


## Transformación de `id_usuario` para la Predicción de Compras

### Objetivo
El objetivo es predecir las futuras compras de cada usuario basándonos en su historial de compras y características adicionales. Para ello, necesitamos decidir cómo tratar la columna `id_usuario` de manera eficiente y efectiva.

### Alternativas Evaluadas

#### 1. Usar `id_usuario` como un número directamente
- **Ventajas**: Simplicidad en la implementación.
- **Desventajas**: El modelo podría interpretar erróneamente los `id_usuario` como valores ordinales, lo cual no tiene sentido en este contexto.

#### 2. One-Hot Encoding
- **Ventajas**: Evita la interpretación ordinal y trata cada `id_usuario` como una categoría única.
- **Desventajas**: Con una alta cardinalidad (2500 usuarios), resulta en una matriz muy dispersa y consume mucha memoria y computación.

#### 3. Label Encoding
- **Ventajas**: Simplicidad y eficiencia en términos de memoria y computación.
- **Desventajas**: Introduce una interpretación ordinal, pero en muchos modelos, esto no afecta significativamente el rendimiento.

#### 4. Embeddings
- **Ventajas**: Captura relaciones complejas entre usuarios y representa `id_usuario` en un espacio de menor dimensión. Es útil para modelos de redes neuronales.
- **Desventajas**: Añade complejidad al modelo y no es necesario si no estamos capturando similitudes entre usuarios.

#### 5. No Incluir `id_usuario`
- **Ventajas**: Simplifica el modelo y evita problemas de interpretación ordinal.
- **Desventajas**: No permite capturar patrones específicos de cada usuario, lo cual es crucial para predicciones personalizadas.

### Análisis y Recomendación

- **Usar `id_usuario` como un número directamente**: No es adecuado porque podría inducir una interpretación ordinal incorrecta.
- **One-Hot Encoding**: No es práctico debido a la alta cardinalidad del `id_usuario`.
- **Label Encoding**: Aunque introduce una interpretación ordinal, es eficiente en términos de memoria y computación. Dado que nuestro objetivo es predecir las futuras compras basándonos únicamente en el historial del usuario, la interpretación ordinal no debería afectar significativamente el rendimiento del modelo.
- **Embeddings**: Añaden complejidad innecesaria dado que no estamos capturando similitudes entre usuarios.
- **No Incluir `id_usuario`**: No permite capturar patrones específicos de cada usuario, lo cual es esencial para predicciones personalizadas.

Por lo tanto, **utilizaremos Label Encoding para `id_usuario`**. Esto nos permitirá manejar eficientemente la alta cardinalidad de los usuarios sin introducir una complejidad innecesaria en el modelo.


In [ ]:
label_encoder = LabelEncoder()
df['id_usuario'] = label_encoder.fit_transform(df['id_usuario'])

In [ ]:
# df = df[df['id_usuario']==1993]

In [ ]:
df['id_usuario'].head()

0    1993
1    4044
2    3455
3    4356
4    3671
Name: id_usuario, dtype: int64

## Escalado de características

### Análisis de Datos

Las características adicionales del dataset son `perfil_digital`, `mes`, `dia`, `dia_semana`, `nse`, y `segmento_unico`. Un análisis descriptivo de estas características muestra que tienen diferentes rangos de valores:

- `perfil_digital`: 0 a 4
- `mes`: 5 a 8
- `dia`: 1 a 31
- `dia_semana`: 0 a
- `nse`: 0 a 3
- `segmento_unico`: 0 a 5

### MinMaxScaler

1. **Rangos Diferentes**: Las características tienen diferentes rangos de valores. Por ejemplo, `dia` varía de 1 a 31, mientras que `nse` varía de 0 a 3. Estas diferencias pueden causar que el modelo trate algunas características como más importantes simplemente por su escala.

2. **Estandarización de Escalas**: Usar MinMaxScaler transformará los valores de todas las características a un rango común (0 a 1). Esto asegura que ninguna característica domine sobre las demás debido a su escala y permite que todas las características contribuyan de manera equitativa al aprendizaje del modelo.

3. **Mejora del Rendimiento del Modelo**: Al escalar las características a un rango uniforme, el modelo puede converger más rápidamente y de manera más estable durante el entrenamiento. Esto puede llevar a una mejora en el rendimiento y la precisión del modelo.


In [ ]:
features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [ ]:
df[features].head()

,perfil_digital,mes,dia,dia_semana,nse,segmento_unico
0,0.50,0.333333,0.000000,0.4,0.333333,0.0
1,0.50,0.333333,0.033333,0.6,0.000000,0.4
2,1.00,0.333333,0.000000,0.4,0.000000,1.0
3,0.25,0.333333,0.000000,0.4,0.000000,0.2
4,1.00,0.333333,0.466667,0.4,0.666667,1.0


## Features y variable objetivo

In [ ]:
# features_total = ['id_usuario'] + features + list(df.filter(regex='^canal_').columns)
# target = df.filter(regex='^producto_').columns

In [ ]:
features_total = features + list(df.filter(regex='^canal_').columns)
target = df.filter(regex='^producto_').columns

In [ ]:
features_total

['perfil_digital',
 'mes',
 'dia',
 'dia_semana',
 'nse',
 'segmento_unico',
 'canal_Autoservicio',
 'canal_BEBIDA',
 'canal_Bar/Restaurant',
 'canal_COMIDA',
 'canal_ENTRETENIMIENTO',
 'canal_Instituciones',
 'canal_KA Minoristas',
 'canal_Kioscos/Maxikioscos',
 'canal_Mayorista',
 'canal_S/D',
 'canal_Tradicional']

In [ ]:
X = df[features_total]
y = df[target]

In [ ]:
X.head()

,perfil_digital,mes,dia,dia_semana,nse,segmento_unico,canal_Autoservicio,canal_BEBIDA,canal_Bar/Restaurant,canal_COMIDA,canal_ENTRETENIMIENTO,canal_Instituciones,canal_KA Minoristas,canal_Kioscos/Maxikioscos,canal_Mayorista,canal_S/D,canal_Tradicional
0,0.50,0.333333,0.000000,0.4,0.333333,0.0,False,False,False,False,False,True,False,False,False,False,False
1,0.50,0.333333,0.033333,0.6,0.000000,0.4,False,False,False,False,False,False,False,False,False,True,False
2,1.00,0.333333,0.000000,0.4,0.000000,1.0,False,False,False,False,False,False,False,False,False,False,True
3,0.25,0.333333,0.000000,0.4,0.000000,0.2,False,False,False,True,False,False,False,False,False,False,False
4,1.00,0.333333,0.466667,0.4,0.666667,1.0,False,False,False,False,False,False,False,False,False,False,True


In [ ]:
y.head()

,producto_270,producto_591,producto_627,producto_641,producto_659,producto_691,producto_808,producto_810,producto_811,producto_813,...,producto_78078,producto_78093,producto_78094,producto_78103,producto_78104,producto_78109,producto_78121,producto_78128,producto_78129,producto_78138
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##División del conjunto de datos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36441, 17), (9111, 17), (36441, 530), (9111, 530))

In [ ]:
y_train = y_train.values
y_test = y_test.values


In [ ]:
X_test = X_test.astype('float32')
X_train = X_train.astype('float32')
y_test = y_test.astype('float32')
y_train = y_train.astype('float32')

## Modelos de Machine Learning para la Predicción de Compras

### Tipo de Problema
El problema que estamos abordando es la predicción de futuras compras de productos por parte de los clientes. Esto puede ser formulado como un problema de clasificación multietiqueta, donde el objetivo es predecir múltiples etiquetas (productos) simultáneamente para cada instancia (cliente). En cada pedido, un cliente puede comprar varios productos, y queremos predecir qué productos serán comprados en futuros pedidos.

### Modelos Seleccionados

#### 1. Random Forest
**Descripción**: Random Forest es un modelo de ensamble basado en la combinación de múltiples árboles de decisión. Cada árbol se entrena con un subconjunto diferente de los datos y las predicciones se hacen mediante el promedio (para regresión) o la mayoría de votos (para clasificación) de todos los árboles.

**Ventajas**:
- Robusto frente al sobreajuste debido a la agregación de múltiples árboles.
- Capaz de manejar características tanto categóricas como numéricas.
- Proporciona una estimación de la importancia de las características.

**Por qué usarlo**: Es un modelo versátil y robusto que puede manejar datasets grandes y complejos, proporcionando buenos resultados en muchas situaciones.

#### 2. XGBoost
**Descripción**: XGBoost (Extreme Gradient Boosting) es un modelo de boosting que crea árboles de decisión secuenciales. Cada árbol intenta corregir los errores de los árboles anteriores, mejorando iterativamente el modelo.

**Ventajas**:
- Alto rendimiento y eficiencia.
- Maneja bien los datos desbalanceados.
- Ofrece técnicas avanzadas de regularización que mejoran el rendimiento y reducen el sobreajuste.

**Por qué usarlo**: Es conocido por su rendimiento superior en competencias de Machine Learning y por su capacidad para manejar datasets grandes y características complejas.

#### 3. Redes Neuronales
**Descripción**: Las Redes Neuronales son modelos inspirados en el cerebro humano que consisten en capas de neuronas. Cada neurona aplica una función de activación a una combinación lineal de sus entradas.

**Ventajas**:
- Capacidad para modelar relaciones no lineales complejas.
- Flexibilidad para ajustarse a diferentes tipos de datos y problemas.
- Potencial para mejorar con grandes cantidades de datos.

**Por qué usarlo**: Son especialmente útiles para capturar patrones complejos en los datos y pueden aprovechar la estructura del dataset para hacer predicciones precisas.

### Conclusión
La combinación de estos tres modelos nos permitirá comparar diferentes enfoques y seleccionar el mejor modelo para predecir las futuras compras de los clientes. Cada modelo tiene sus propias fortalezas y puede ofrecer perspectivas únicas sobre los datos.

#Primeras aproximaciones para evaluar resultados

##Random Forest

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=5, max_depth=7, random_state=42)
multi_target_regressor = MultiOutputRegressor(rf_regressor, n_jobs=-1)

In [ ]:
multi_target_regressor.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
y_pred_regressor = multi_target_regressor.predict(X_test)

In [ ]:
np.where(y_pred_regressor[3] >= 0.4)

In [ ]:
np.where(y_test[3] ==1)

##XGBoost

In [ ]:
xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, max_depth=7, eval_metric='logloss', random_state=42)

In [ ]:
multi_target_xgb = MultiOutputClassifier(xgb_classifier, n_jobs=-1)

In [ ]:
multi_target_xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = multi_target_xgb.predict(X_test)

In [ ]:
np.where(y_pred_xgb[3] >= 0.4)

In [ ]:
np.where(y_test[3] ==1)

##Redes Neuronales

In [ ]:
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

In [ ]:
model = Sequential([Dense(64, input_dim=input_dim, activation='relu'),
        Dense(32, activation='relu'),
        Dense(output_dim, activation='sigmoid')])


In [ ]:
model.compile(optimizer=Nadam(learning_rate=0.001), loss='binary_crossentropy', metrics='accuracy')

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test), verbose=1)

In [ ]:
y_pred_nn = model.predict(X_test)

In [ ]:
np.where(y_pred_nn[3] >= 0.4)

In [ ]:
np.where(y_test[3] ==1)

In [ ]:
comparaciones = {'y_test':[], 'y_pred_regressor':[], 'y_pred_xgb':[], 'y_pred_nn':[]}

In [ ]:
for i in range(len(y_test)):
    comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
    comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
    comparaciones['y_pred_xgb'].append(np.where(y_pred_xgb[i] >= 0.4)[0])
    comparaciones['y_pred_nn'].append(np.where(y_pred_nn[i] >= 0.4)[0])

In [ ]:
df_comparaciones = pd.DataFrame(comparaciones)
df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
df_comparaciones['coincidencias_xgb'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_xgb'])), axis=1)
df_comparaciones['coincidencias_nn'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_nn'])), axis=1)
df_comparaciones = df_comparaciones[['y_test', 'y_pred_regressor', 'coincidencias_regressor', 'y_pred_xgb', 'coincidencias_xgb', 'y_pred_nn', 'coincidencias_nn']]
df_comparaciones

In [ ]:
def modelos_usuario(data:pd.DataFrame, usuario:int) -> None:
    print(f'\n\nUsuario: {usuario}\n\n')
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_train = y_train.values
    y_test = y_test.values
    X_test = X_test.astype('float32')
    X_train = X_train.astype('float32')
    y_test = y_test.astype('float32')
    y_train = y_train.astype('float32')
    rf_regressor = RandomForestRegressor(n_estimators=60, max_depth=7, random_state=42)
    multi_target_regressor = MultiOutputRegressor(rf_regressor, n_jobs=-1)
    multi_target_regressor.fit(X_train, y_train)
    y_pred_regressor = multi_target_regressor.predict(X_test)
    xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, max_depth=7, eval_metric='logloss', random_state=42)
    multi_target_xgb = MultiOutputClassifier(xgb_classifier, n_jobs=-1)
    multi_target_xgb.fit(X_train, y_train)
    y_pred_xgb = multi_target_xgb.predict(X_test)
    input_dim = X_train.shape[1]
    output_dim = y_train.shape[1]
    model = Sequential([Dense(256, input_dim=input_dim, activation='relu'),
            Dense(64, activation='relu'),
            Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
            Dense(32, activation='relu'),
            Dense(output_dim, activation='sigmoid')])
    model.compile(optimizer=Nadam(learning_rate=0.01), loss='binary_crossentropy', metrics='accuracy')
    model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test), verbose=0)
    y_pred_nn = model.predict(X_test)
    comparaciones = {'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[], 'y_pred_xgb':[], 'productos_xgb':[], 'y_pred_nn':[], 'productos_nn':[]}
    for i in range(len(y_test)):
        comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
        comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
        comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
        comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
        comparaciones['y_pred_xgb'].append(np.where(y_pred_xgb[i] >= 0.4)[0])
        comparaciones['productos_xgb'].append(len(np.where(y_pred_xgb[i] >= 0.4)[0]))
        comparaciones['y_pred_nn'].append(np.where(y_pred_nn[i] >= 0.4)[0])
        comparaciones['productos_nn'].append(len(np.where(y_pred_nn[i] >= 0.4)[0]))
    df_comparaciones = pd.DataFrame(comparaciones)
    df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
    df_comparaciones['coincidencias_xgb'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_xgb'])), axis=1)
    df_comparaciones['coincidencias_nn'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_nn'])), axis=1)
    df_comparaciones = df_comparaciones[['y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor', 'y_pred_xgb', 'productos_xgb', 'coincidencias_xgb', 'y_pred_nn', 'productos_nn', 'coincidencias_nn']]
    print(df_comparaciones)


In [ ]:
usuarios = [randint(1, 2500) for _ in range(5)]
usuarios

[990, 1297, 2080, 2354, 468]

In [ ]:
inicio = time.time()
modelos_usuario(df, 1993)
final = time.time()
print(f'Tiempo de ejecución: {final - inicio} segundos')



Usuario: 1993


1/1 [==============================] - 0s 67ms/step
                   y_test  productos_test                   y_pred_regressor  \
0  [20, 26, 92, 196, 337]               5  [26, 92, 156, 200, 261, 337, 338]   
1       [20, 26, 91, 200]               4           [26, 156, 200, 261, 348]   
2          [92, 110, 465]               3  [26, 92, 156, 200, 337, 338, 348]   
3               [26, 200]               2            [26, 92, 200, 337, 338]   

   productos_regressor  coincidencias_regressor  y_pred_xgb  productos_xgb  \
0                    7                        3  [156, 200]              2   
1                    5                        2  [156, 200]              2   
2                    7                        1  [156, 200]              2   
3                    5                        2       [200]              1   

   coincidencias_xgb           y_pred_nn  productos_nn  coincidencias_nn  
0                  0  [26, 92, 196, 200]             4         

In [ ]:
for usuario in usuarios:
    try:
        modelos_usuario(df, usuario)
        print(10*'\n')
    except:
        continue



Usuario: 922


1/1 [==============================] - 0s 76ms/step
            y_test y_pred_regressor  coincidencias_regressor y_pred_xgb  \
0  [149, 160, 200]   [48, 149, 167]                        1         []   

   coincidencias_xgb   y_pred_nn  coincidencias_nn  
0                  0  [149, 167]                 1  













Usuario: 919


1/1 [==============================] - 0s 79ms/step
           y_test   y_pred_regressor  coincidencias_regressor y_pred_xgb  \
0  [48, 293, 485]      [44, 48, 293]                        2       [48]   
1            [44]          [48, 293]                        0       [48]   
2        [44, 48]  [44, 48, 62, 389]                        2   [44, 48]   

   coincidencias_xgb           y_pred_nn  coincidencias_nn  
0                  1       [44, 48, 293]                 2  
1                  0  [44, 48, 293, 389]                 1  
2                  2       [44, 48, 293]                 2  













Usuario: 1823


1/1 [===========

In [ ]:
usuarios = [randint(1, 4000) for _ in range(10)]
usuarios

[609, 3783, 756, 3201, 3553, 1229, 545, 22, 3142, 2842]

In [ ]:
for usuario in usuarios:
    try:
        modelos_usuario(df, usuario)
        print(10*'\n')
    except:
        continue



Usuario: 609


1/1 [==============================] - 0s 65ms/step
                y_test  productos_test  \
0                 [48]               1   
1  [44, 116, 117, 150]               4   
2      [149, 160, 293]               3   
3       [48, 146, 387]               3   

                                    y_pred_regressor  productos_regressor  \
0  [31, 43, 48, 62, 89, 135, 149, 150, 159, 293, ...                   11   
1                                     [45, 293, 389]                    3   
2  [43, 45, 55, 56, 59, 63, 94, 152, 157, 221, 26...                   13   
3  [1, 31, 43, 55, 56, 59, 62, 63, 92, 94, 116, 1...                   21   

   coincidencias_regressor y_pred_xgb  productos_xgb  coincidencias_xgb  \
0                        1       [48]              1                  1   
1                        0  [45, 389]              2                  0   
2                        0   [45, 56]              2                  0   
3                        0       [

In [ ]:
usuarios = [randint(1, 4000) for _ in range(10)]
usuarios

[2743, 1473, 749, 2001, 2318, 1444, 963, 1784, 2051, 864]

In [ ]:
for usuario in usuarios:
    try:
        modelos_usuario(df, usuario)
        print(10*'\n')
    except:
        continue



Usuario: 2743




Usuario: 1473




Usuario: 749




Usuario: 2001




Usuario: 2318




Usuario: 1444




Usuario: 963


1/1 [==============================] - 0s 139ms/step
                   y_test  productos_test  \
0  [26, 53, 55, 156, 157]               5   

                         y_pred_regressor  productos_regressor  \
0  [26, 51, 52, 54, 55, 56, 65, 157, 404]                    9   

   coincidencias_regressor        y_pred_xgb  productos_xgb  \
0                        3  [51, 52, 54, 55]              4   

   coincidencias_xgb                  y_pred_nn  productos_nn  \
0                  1  [51, 52, 53, 54, 55, 404]             6   

   coincidencias_nn  
0                 2  













Usuario: 1784




Usuario: 2051


1/1 [==============================] - 0s 146ms/step
                                              y_test  productos_test  \
0  [18, 19, 62, 80, 96, 119, 129, 131, 132, 143, ...              17   
1  [44, 48, 62, 96, 113, 119, 129, 147, 149, 157,... 

In [ ]:
usuarios = [randint(1, 4000) for _ in range(10)]
usuarios

[1800, 3437, 2570, 929, 217, 2020, 2747, 507, 1535, 1966]

In [ ]:
for usuario in usuarios:
    try:
        modelos_usuario(df, usuario)
        print(10*'\n')
    except:
        continue



Usuario: 1800


1/1 [==============================] - 0s 226ms/step
                              y_test  productos_test  \
0  [48, 96, 116, 149, 160, 261, 293]               7   
1           [48, 149, 160, 320, 370]               5   

                                 y_pred_regressor  productos_regressor  \
0  [18, 44, 61, 62, 149, 160, 261, 268, 320, 370]                   10   
1                         [48, 96, 149, 160, 261]                    5   

   coincidencias_regressor           y_pred_xgb  productos_xgb  \
0                        3  [48, 149, 261, 293]              4   
1                        3  [48, 149, 261, 293]              4   

   coincidencias_xgb                         y_pred_nn  productos_nn  \
0                  4  [18, 48, 62, 149, 160, 261, 293]             7   
1                  2  [18, 48, 62, 149, 160, 261, 293]             7   

   coincidencias_nn  
0                 5  
1                 3  













Usuario: 3437


1/1 [=====================

1/1 [==============================] - 0s 59ms/step
                        y_test  productos_test  \
0  [43, 46, 48, 132, 146, 365]               6   
1                    [42, 370]               2   

                    y_pred_regressor  productos_regressor  \
0  [62, 89, 146, 161, 268, 365, 502]                    7   
1  [62, 89, 146, 161, 268, 365, 502]                    7   

   coincidencias_regressor y_pred_xgb  productos_xgb  coincidencias_xgb  \
0                        2      [268]              1                  0   
1                        0      [268]              1                  0   

  y_pred_nn  productos_nn  coincidencias_nn  
0     [268]             1                 0  
1     [268]             1                 0  













Usuario: 2747




1/1 [==============================] - 0s 62ms/step
                    y_test  productos_test               y_pred_regressor  \
0      [62, 161, 392, 472]               4  [48, 117, 294, 367, 389, 428]   
1  [48, 63, 115, 116, 293]               5                      [48, 389]   
2                [42, 180]               2                  [43, 62, 428]   

   productos_regressor  coincidencias_regressor y_pred_xgb  productos_xgb  \
0                    6                        0         []              0   
1                    2                        1         []              0   
2                    3                        0         []              0   

   coincidencias_xgb y_pred_nn  productos_nn  coincidencias_nn  
0                  0      [48]             1                 0  
1                  0      [48]             1                 1  
2                  0      [48]             1                 0  













Usuario: 507


1/1 [==============================] - 0s 

In [ ]:
def modelos_usuario_metricas(data:pd.DataFrame, usuario:int) -> None:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_train = y_train.values
    y_test = y_test.values
    X_test = X_test.astype('float32')
    X_train = X_train.astype('float32')
    y_test = y_test.astype('float32')
    y_train = y_train.astype('float32')
    rf_regressor = RandomForestRegressor(n_estimators=60, max_depth=7, random_state=42)
    multi_target_regressor = MultiOutputRegressor(rf_regressor, n_jobs=-1)
    multi_target_regressor.fit(X_train, y_train)
    y_pred_regressor = multi_target_regressor.predict(X_test)
    xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, max_depth=7, eval_metric='logloss', random_state=42)
    multi_target_xgb = MultiOutputClassifier(xgb_classifier, n_jobs=-1)
    multi_target_xgb.fit(X_train, y_train)
    y_pred_xgb = multi_target_xgb.predict(X_test)
    input_dim = X_train.shape[1]
    output_dim = y_train.shape[1]
    model = Sequential([Dense(256, input_dim=input_dim, activation='relu'),
            Dense(64, activation='relu'),
            Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
            Dense(32, activation='relu'),
            Dense(output_dim, activation='sigmoid')])
    model.compile(optimizer=Nadam(learning_rate=0.01), loss='binary_crossentropy', metrics='accuracy')
    model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test), verbose=0)
    y_pred_nn = model.predict(X_test)
    comparaciones = {'usuario':[],'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[], 'y_pred_xgb':[], 'productos_xgb':[], 'y_pred_nn':[], 'productos_nn':[]}
    for i in range(len(y_test)):
        comparaciones['usuario'].append(usuario)
        comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
        comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
        comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
        comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
        comparaciones['y_pred_xgb'].append(np.where(y_pred_xgb[i] >= 0.4)[0])
        comparaciones['productos_xgb'].append(len(np.where(y_pred_xgb[i] >= 0.4)[0]))
        comparaciones['y_pred_nn'].append(np.where(y_pred_nn[i] >= 0.4)[0])
        comparaciones['productos_nn'].append(len(np.where(y_pred_nn[i] >= 0.4)[0]))
    df_comparaciones = pd.DataFrame(comparaciones)
    df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
    df_comparaciones['coincidencias_xgb'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_xgb'])), axis=1)
    df_comparaciones['coincidencias_nn'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_nn'])), axis=1)
    df_comparaciones = df_comparaciones[['usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor', 'y_pred_xgb', 'productos_xgb', 'coincidencias_xgb', 'y_pred_nn', 'productos_nn', 'coincidencias_nn']]
    return df_comparaciones


In [ ]:
df_comparaciones = pd.DataFrame()

In [ ]:
df_comparaciones

""


In [ ]:
df_1 = modelos_usuario_metricas(df, 1993)


1/1 [==============================] - 0s 201ms/step


In [ ]:
df_1

,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor,y_pred_xgb,productos_xgb,coincidencias_xgb,y_pred_nn,productos_nn,coincidencias_nn
0,1993,"[20, 26, 92, 196, 337]",5,"[26, 92, 156, 200, 261, 337, 338]",7,3,"[156, 200]",2,0,"[92, 196, 200]",3,2
1,1993,"[20, 26, 91, 200]",4,"[26, 156, 200, 261, 348]",5,2,"[156, 200]",2,1,"[92, 196, 200]",3,1
2,1993,"[92, 110, 465]",3,"[26, 92, 156, 200, 337, 338, 348]",7,1,"[156, 200]",2,0,"[92, 196, 200]",3,1
3,1993,"[26, 200]",2,"[26, 92, 200, 337, 338]",5,2,[200],1,1,"[92, 196, 200]",3,1


In [ ]:
df_comparaciones = pd.concat([df_comparaciones, df_1])

In [ ]:
df_comparaciones

,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor,y_pred_xgb,productos_xgb,coincidencias_xgb,y_pred_nn,productos_nn,coincidencias_nn
0,1993,"[20, 26, 92, 196, 337]",5,"[26, 92, 156, 200, 261, 337, 338]",7,3,"[156, 200]",2,0,"[92, 196, 200]",3,2
1,1993,"[20, 26, 91, 200]",4,"[26, 156, 200, 261, 348]",5,2,"[156, 200]",2,1,"[92, 196, 200]",3,1
2,1993,"[92, 110, 465]",3,"[26, 92, 156, 200, 337, 338, 348]",7,1,"[156, 200]",2,0,"[92, 196, 200]",3,1
3,1993,"[26, 200]",2,"[26, 92, 200, 337, 338]",5,2,[200],1,1,"[92, 196, 200]",3,1


In [ ]:
df_1 = modelos_usuario_metricas(df, 537)


1/1 [==============================] - 0s 88ms/step


In [ ]:
df_comparaciones = pd.concat([df_comparaciones, df_1])

In [ ]:
df_comparaciones

,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor,y_pred_xgb,productos_xgb,coincidencias_xgb,y_pred_nn,productos_nn,coincidencias_nn
0,1993,"[20, 26, 92, 196, 337]",5,"[26, 92, 156, 200, 261, 337, 338]",7,3,"[156, 200]",2,0,"[196, 200]",2,1
1,1993,"[20, 26, 91, 200]",4,"[26, 156, 200, 261, 348]",5,2,"[156, 200]",2,1,"[196, 200]",2,1
2,1993,"[92, 110, 465]",3,"[26, 92, 156, 200, 337, 338, 348]",7,1,"[156, 200]",2,0,"[196, 200]",2,0
3,1993,"[26, 200]",2,"[26, 92, 200, 337, 338]",5,2,[200],1,1,"[196, 200]",2,1
0,537,"[3, 39, 91]",3,"[91, 196]",2,1,"[91, 196]",2,1,"[91, 196]",2,1
1,537,"[39, 91, 196]",3,"[91, 196]",2,2,"[91, 196]",2,2,"[91, 196]",2,2


In [ ]:
df_comparaciones = pd.DataFrame()

In [ ]:
usuarios = [randint(1, 4000) for _ in range(30)]
usuarios

[1068,
 497,
 2594,
 1209,
 279,
 3752,
 3811,
 2878,
 335,
 3822,
 1091,
 3525,
 3797,
 2608,
 505,
 199,
 1215,
 1614,
 3773,
 1265,
 105,
 3233,
 3249,
 86,
 3596,
 15,
 2348,
 2775,
 2386,
 1268]

In [ ]:
for usuario in usuarios:
    try:
        if usuario not in df_comparaciones['usuario'].unique():
            df_1 = modelos_usuario_metricas(df, usuario)
            df_comparaciones = pd.concat([df_comparaciones, df_1])
        else:
            continue
    except:
        continue
ruta_excel = comparaciones + '/comparaciones_04.xlsx'
df_comparaciones.to_excel(ruta_excel, index=False)

1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 68ms/step


1/1 [==============================] - 0s 132ms/step


In [ ]:
df_comparaciones

,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor,y_pred_xgb,productos_xgb,coincidencias_xgb,y_pred_nn,productos_nn,coincidencias_nn
0,1993,"[20, 26, 92, 196, 337]",5,"[26, 92, 156, 200, 261, 337, 338]",7,3,"[156, 200]",2,0,"[92, 196, 200]",3,2
1,1993,"[20, 26, 91, 200]",4,"[26, 156, 200, 261, 348]",5,2,"[156, 200]",2,1,"[92, 196, 200]",3,1
2,1993,"[92, 110, 465]",3,"[26, 92, 156, 200, 337, 338, 348]",7,1,"[156, 200]",2,0,"[92, 196, 200]",3,1
3,1993,"[26, 200]",2,"[26, 92, 200, 337, 338]",5,2,[200],1,1,"[92, 196, 200]",3,1
0,2594,"[31, 62, 247, 268, 307, 320]",6,"[48, 62, 108, 117, 154, 161, 184, 247, 293, 30...",12,2,"[48, 293, 389]",3,0,"[48, 62, 161, 293, 356]",5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2,2348,"[10, 261]",2,"[6, 10, 261, 262]",4,2,[],0,0,"[6, 261]",2,1
0,2775,"[31, 62, 96, 103, 149, 160, 268, 301]",8,"[32, 62, 96, 149, 160, 184, 268, 336]",8,5,"[96, 149, 160]",3,3,"[96, 149, 160]",3,3
1,2775,"[96, 149, 160, 268, 336, 472, 494, 503]",8,"[96, 149, 160, 268, 336, 337]",6,5,"[96, 149, 160]",3,3,"[96, 149, 160]",3,3
0,1268,"[131, 132, 149, 160, 317]",5,[],0,0,[],0,0,[62],1,0


In [ ]:
"""
TP = coincidencias_regressor
FP = productos_regressor - coincidencias_regressor
FN = productos_test - coincidencias_regressor
TN = 530 - productos_test - productos_regressor + coincidencias_regressor
"""

In [ ]:
lista = []
for usuario in df_comparaciones['usuario'].unique():
    diccionario = {}
    diccionario['usuario'] = usuario
    df_usuario = df_comparaciones[df_comparaciones['usuario']==usuario]
    diccionario['TP_regressor'] = df_usuario['coincidencias_regressor'].sum()
    diccionario['FP_regressor'] = df_usuario['productos_regressor'].sum() - df_usuario['coincidencias_regressor'].sum()
    diccionario['FN_regressor'] = df_usuario['productos_test'].sum() - df_usuario['coincidencias_regressor'].sum()
    diccionario['TN_regressor'] = 530 - df_usuario['productos_test'].sum() - df_usuario['productos_regressor'].sum() + df_usuario['coincidencias_regressor'].sum()
    diccionario['TP_xgb'] = df_usuario['coincidencias_xgb'].sum()
    diccionario['FP_xgb'] = df_usuario['productos_xgb'].sum() - df_usuario['coincidencias_xgb'].sum()
    diccionario['FN_xgb'] = df_usuario['productos_test'].sum() - df_usuario['coincidencias_xgb'].sum()
    diccionario['TN_xgb'] = 530 - df_usuario['productos_test'].sum() - df_usuario['productos_xgb'].sum() + df_usuario['coincidencias_xgb'].sum()
    diccionario['TP_nn'] = df_usuario['coincidencias_nn'].sum()
    diccionario['FP_nn'] = df_usuario['productos_nn'].sum() - df_usuario['coincidencias_nn'].sum()
    diccionario['FN_nn'] = df_usuario['productos_test'].sum() - df_usuario['coincidencias_nn'].sum()
    diccionario['TN_nn'] = 530 - df_usuario['productos_test'].sum() - df_usuario['productos_nn'].sum() + df_usuario['coincidencias_nn'].sum()

    lista.append(diccionario)



In [ ]:
lista

[{'usuario': 1993,
  'TP_regressor': 8,
  'FP_regressor': 16,
  'FN_regressor': 6,
  'TN_regressor': 500,
  'TP_xgb': 2,
  'FP_xgb': 5,
  'FN_xgb': 12,
  'TN_xgb': 511,
  'TP_nn': 5,
  'FP_nn': 7,
  'FN_nn': 9,
  'TN_nn': 509},
 {'usuario': 2594,
  'TP_regressor': 17,
  'FP_regressor': 48,
  'FN_regressor': 23,
  'TN_regressor': 442,
  'TP_xgb': 5,
  'FP_xgb': 12,
  'FN_xgb': 35,
  'TN_xgb': 478,
  'TP_nn': 5,
  'FP_nn': 16,
  'FN_nn': 35,
  'TN_nn': 474},
 {'usuario': 1209,
  'TP_regressor': 9,
  'FP_regressor': 48,
  'FN_regressor': 33,
  'TN_regressor': 440,
  'TP_xgb': 6,
  'FP_xgb': 7,
  'FN_xgb': 36,
  'TN_xgb': 481,
  'TP_nn': 6,
  'FP_nn': 4,
  'FN_nn': 36,
  'TN_nn': 484},
 {'usuario': 279,
  'TP_regressor': 2,
  'FP_regressor': 10,
  'FN_regressor': 7,
  'TN_regressor': 511,
  'TP_xgb': 0,
  'FP_xgb': 0,
  'FN_xgb': 9,
  'TN_xgb': 521,
  'TP_nn': 0,
  'FP_nn': 3,
  'FN_nn': 9,
  'TN_nn': 518},
 {'usuario': 3752,
  'TP_regressor': 4,
  'FP_regressor': 6,
  'FN_regressor': 4,
 

In [ ]:
for usuario in lista:
    precision_regressor = usuario['TP_regressor'] / (usuario['TP_regressor'] + usuario['FP_regressor'])
    recall_regressor = usuario['TP_regressor'] / (usuario['TP_regressor'] + usuario['FN_regressor'])
    f1_score_regressor = 2 * (precision_regressor * recall_regressor) / (precision_regressor + recall_regressor)
    precision_xgb = usuario['TP_xgb'] / (usuario['TP_xgb'] + usuario['FP_xgb'])
    recall_xgb = usuario['TP_xgb'] / (usuario['TP_xgb'] + usuario['FN_xgb'])
    f1_score_xgb = 2 * (precision_xgb * recall_xgb) / (precision_xgb + recall_xgb)
    precision_nn = usuario['TP_nn'] / (usuario['TP_nn'] + usuario['FP_nn'])
    recall_nn = usuario['TP_nn'] / (usuario['TP_nn'] + usuario['FN_nn'])
    f1_score_nn = 2 * (precision_nn * recall_nn) / (precision_nn + recall_nn)
    print(f'Usuario: {usuario["usuario"]}')
    print(f'Precision_regressor: {round(precision_regressor*100,2)}, Precision_xgb: {round(precision_xgb*100,2)}, Precision_nn: {round(precision_nn*100,2)}')
    print(f'recall_regressor: {round(recall_regressor*100,2)}, recall_xgb: {round(recall_xgb*100,2)}, recall_nn: {round(recall_nn*100,2)}')
    print(f'f1_score_regressor: {round(f1_score_regressor,2)}, f1_score_xgb: {round(f1_score_xgb,2)}, f1_score_nn: {round(f1_score_nn,2)}')
    print('\n\n\n')

Usuario: 1993
Precision_regressor: 33.33, Precision_xgb: 28.57, Precision_nn: 41.67
recall_regressor: 57.14, recall_xgb: 14.29, recall_nn: 35.71
f1_score_regressor: 0.42, f1_score_xgb: 0.19, f1_score_nn: 0.38




Usuario: 2594
Precision_regressor: 26.15, Precision_xgb: 29.41, Precision_nn: 23.81
recall_regressor: 42.5, recall_xgb: 12.5, recall_nn: 12.5
f1_score_regressor: 0.32, f1_score_xgb: 0.18, f1_score_nn: 0.16




Usuario: 1209
Precision_regressor: 15.79, Precision_xgb: 46.15, Precision_nn: 60.0
recall_regressor: 21.43, recall_xgb: 14.29, recall_nn: 14.29
f1_score_regressor: 0.18, f1_score_xgb: 0.22, f1_score_nn: 0.23




Usuario: 279
Precision_regressor: 16.67, Precision_xgb: nan, Precision_nn: 0.0
recall_regressor: 22.22, recall_xgb: 0.0, recall_nn: 0.0
f1_score_regressor: 0.19, f1_score_xgb: nan, f1_score_nn: nan




Usuario: 3752
Precision_regressor: 40.0, Precision_xgb: 100.0, Precision_nn: 100.0
recall_regressor: 50.0, recall_xgb: 50.0, recall_nn: 50.0
f1_score_regressor: 0.

In [ ]:
def evaluuacion_sin_hiper(data:pd.DataFrame, cantidad:int) -> pd.DataFrame:
    df_comparaciones = pd.DataFrame()
    usuarios = [randint(1, 4000) for _ in range(cantidad)]
    for usuario in usuarios:
        try:
            df_1 = modelos_usuario_metricas(data, usuario)
            df_comparaciones = pd.concat([df_comparaciones, df_1])
        except:
            continue
    ruta_excel = comparaciones + '/comparaciones_04.xlsx'
    df_comparaciones.to_excel(ruta_excel, index=False)
    return df_comparaciones


In [ ]:
df_comparaciones_2 = evaluuacion_sin_hiper(df, 50)

1/1 [==============================] - 0s 97ms/step


1/1 [==============================] - 0s 115ms/step


1/1 [==============================] - 0s 73ms/step
Usuario: 2490
Precision_regressor: 0    0.0
1    0.0
2    0.0
dtype: float64, Precision_xgb: 0   NaN
1   NaN
2   NaN
dtype: float64, Precision_nn: 0   NaN
1   NaN
2   NaN
dtype: float64
recall_regressor: 0    0.0
1    0.0
2    0.0
dtype: float64, recall_xgb: 0    0.0
1    0.0
2    0.0
dtype: float64, recall_nn: 0    0.0
1    0.0
2    0.0
dtype: float64
f1_score_regressor: 0   NaN
1   NaN
2   NaN
dtype: float64, f1_score_xgb: 0   NaN
1   NaN
2   NaN
dtype: float64, f1_score_nn: 0   NaN
1   NaN
2   NaN
dtype: float64






In [ ]:
def metricas(data:pd.DataFrame) -> None:
    lista = []
    for usuario in data['usuario'].unique():
        diccionario = {}
        diccionario['usuario'] = usuario
        df_usuario = data[data['usuario']==usuario]
        diccionario['TP_regressor'] = df_usuario['coincidencias_regressor'].sum()
        diccionario['FP_regressor'] = df_usuario['productos_regressor'].sum() - df_usuario['coincidencias_regressor'].sum()
        diccionario['FN_regressor'] = df_usuario['productos_test'].sum() - df_usuario['coincidencias_regressor'].sum()
        diccionario['TN_regressor'] = 530 - df_usuario['productos_test'].sum() - df_usuario['productos_regressor'].sum() + df_usuario['coincidencias_regressor'].sum()
        diccionario['TP_xgb'] = df_usuario['coincidencias_xgb'].sum()
        diccionario['FP_xgb'] = df_usuario['productos_xgb'].sum() - df_usuario['coincidencias_xgb'].sum()
        diccionario['FN_xgb'] = df_usuario['productos_test'].sum() - df_usuario['coincidencias_xgb'].sum()
        diccionario['TN_xgb'] = 530 - df_usuario['productos_test'].sum() - df_usuario['productos_xgb'].sum() + df_usuario['coincidencias_xgb'].sum()
        diccionario['TP_nn'] = df_usuario['coincidencias_nn'].sum()
        diccionario['FP_nn'] = df_usuario['productos_nn'].sum() - df_usuario['coincidencias_nn'].sum()
        diccionario['FN_nn'] = df_usuario['productos_test'].sum() - df_usuario['coincidencias_nn'].sum()
        diccionario['TN_nn'] = 530 - df_usuario['productos_test'].sum() - df_usuario['productos_nn'].sum() + df_usuario['coincidencias_nn'].sum()
        lista.append(diccionario)
    precision_regressor_ = []
    precision_xgb_ = []
    precision_nn_ = []
    recall_regressor_ = []
    recall_xgb_ = []
    recall_nn_ = []
    f1_score_regressor_ = []
    f1_score_xgb_ = []
    f1_score_nn_ = []
    for usuario in lista:
        precision_regressor = usuario['TP_regressor'] / (usuario['TP_regressor'] + usuario['FP_regressor'])
        recall_regressor = usuario['TP_regressor'] / (usuario['TP_regressor'] + usuario['FN_regressor'])
        f1_score_regressor = 2 * (precision_regressor * recall_regressor) / (precision_regressor + recall_regressor)
        precision_xgb = usuario['TP_xgb'] / (usuario['TP_xgb'] + usuario['FP_xgb'])
        recall_xgb = usuario['TP_xgb'] / (usuario['TP_xgb'] + usuario['FN_xgb'])
        f1_score_xgb = 2 * (precision_xgb * recall_xgb) / (precision_xgb + recall_xgb)
        precision_nn = usuario['TP_nn'] / (usuario['TP_nn'] + usuario['FP_nn'])
        recall_nn = usuario['TP_nn'] / (usuario['TP_nn'] + usuario['FN_nn'])
        f1_score_nn = 2 * (precision_nn * recall_nn) / (precision_nn + recall_nn)
        precision_regressor_.append(precision_regressor if not np.isnan(precision_regressor) else 0)
        precision_xgb_.append(precision_xgb if not np.isnan(precision_xgb) else 0)
        precision_nn_.append(precision_nn if not np.isnan(precision_nn) else 0)
        recall_regressor_.append(recall_regressor if not np.isnan(recall_regressor) else 0)
        recall_xgb_.append(recall_xgb if not np.isnan(recall_xgb) else 0)
        recall_nn_.append(recall_nn if not np.isnan(recall_nn) else 0)
        f1_score_regressor_.append(f1_score_regressor if not np.isnan(f1_score_regressor) else 0)
        f1_score_xgb_.append(f1_score_xgb if not np.isnan(f1_score_xgb) else 0)
        f1_score_nn_.append(f1_score_nn if not np.isnan(f1_score_nn) else 0)
        print(f'Usuario: {usuario["usuario"]}')
        print(f'Precision_regressor: {round(precision_regressor*100,2)}, Precision_xgb: {round(precision_xgb*100,2)}, Precision_nn: {round(precision_nn*100,2)}')
        print(f'recall_regressor: {round(recall_regressor*100,2)}, recall_xgb: {round(recall_xgb*100,2)}, recall_nn: {round(recall_nn*100,2)}')
        print(f'f1_score_regressor: {round(f1_score_regressor,2)}, f1_score_xgb: {round(f1_score_xgb,2)}, f1_score_nn: {round(f1_score_nn,2)}')
        print('\n')
    print('Promedios:')
    print(f'precision_regressor: {round(sum(precision_regressor_)/len(precision_regressor_)*100,2)}')
    print(f'precision_xgb: {round(sum(precision_xgb_)/len(precision_xgb_)*100,2)}')
    print(f'precision_nn: {round(sum(precision_nn_)/len(precision_nn_)*100,2)}')
    print(f'recall_regressor: {round(sum(recall_regressor_)/len(recall_regressor_)*100,2)}')
    print(f'recall_xgb: {round(sum(recall_xgb_)/len(recall_xgb_)*100,2)}')
    print(f'recall_nn: {round(sum(recall_nn_)/len(recall_nn_)*100,2)}')
    print(f'f1_score_regressor: {round(sum(f1_score_regressor_)/len(f1_score_regressor_),2)}')
    print(f'f1_score_xgb: {round(sum(f1_score_xgb_)/len(f1_score_xgb_),2)}')
    print(f'f1_score_nn: {round(sum(f1_score_nn_)/len(f1_score_nn_),2)}')




In [ ]:
metricas(df_comparaciones_2)

Usuario: 2293
Precision_regressor: 20.0, Precision_xgb: nan, Precision_nn: 33.33
recall_regressor: 16.67, recall_xgb: 0.0, recall_nn: 33.33
f1_score_regressor: 0.18, f1_score_xgb: nan, f1_score_nn: 0.33


Usuario: 2460
Precision_regressor: 16.13, Precision_xgb: 33.33, Precision_nn: 33.33
recall_regressor: 16.67, recall_xgb: 6.67, recall_nn: 6.67
f1_score_regressor: 0.16, f1_score_xgb: 0.11, f1_score_nn: 0.11


Usuario: 62
Precision_regressor: 33.33, Precision_xgb: 50.0, Precision_nn: 50.0
recall_regressor: 27.27, recall_xgb: 9.09, recall_nn: 45.45
f1_score_regressor: 0.3, f1_score_xgb: 0.15, f1_score_nn: 0.48


Usuario: 3134
Precision_regressor: 20.0, Precision_xgb: nan, Precision_nn: 30.0
recall_regressor: 16.67, recall_xgb: 0.0, recall_nn: 50.0
f1_score_regressor: 0.18, f1_score_xgb: nan, f1_score_nn: 0.37


Usuario: 944
Precision_regressor: 4.76, Precision_xgb: 0.0, Precision_nn: nan
recall_regressor: 5.0, recall_xgb: 0.0, recall_nn: 0.0
f1_score_regressor: 0.05, f1_score_xgb: nan, 

In [ ]:
def modelos_usuario_metricas_ultimo_registro(data:pd.DataFrame, usuario:int) -> pd.Dataframe:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    if len(data) == 1:
        return y
    else:
        X_train = X.iloc[:-1,:]
        y_train = y.iloc[:-1,:]
        X_test = X.iloc[-1:,:]
        y_test = y.iloc[-1:,:]
        y_train = y_train.values
        y_test = y_test.values
        X_test = X_test.astype('float32')
        X_train = X_train.astype('float32')
        y_test = y_test.astype('float32')
        y_train = y_train.astype('float32')
        rf_regressor = RandomForestRegressor(n_estimators=60, max_depth=7, random_state=42)
        multi_target_regressor = MultiOutputRegressor(rf_regressor, n_jobs=-1)
        multi_target_regressor.fit(X_train, y_train)
        y_pred_regressor = multi_target_regressor.predict(X_test)
        xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, max_depth=7, eval_metric='logloss', random_state=42)
        multi_target_xgb = MultiOutputClassifier(xgb_classifier, n_jobs=-1)
        multi_target_xgb.fit(X_train, y_train)
        y_pred_xgb = multi_target_xgb.predict(X_test)
        input_dim = X_train.shape[1]
        output_dim = y_train.shape[1]
        model = Sequential([Dense(256, input_dim=input_dim, activation='relu'),
                Dense(64, activation='relu'),
                Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
                Dense(32, activation='relu'),
                Dense(output_dim, activation='sigmoid')])
        model.compile(optimizer=Nadam(learning_rate=0.01), loss='binary_crossentropy', metrics='accuracy')
        model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test), verbose=0)
        y_pred_nn = model.predict(X_test)
        comparaciones = {'usuario':[],'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[], 'y_pred_xgb':[], 'productos_xgb':[], 'y_pred_nn':[], 'productos_nn':[]}
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
            comparaciones['y_pred_xgb'].append(np.where(y_pred_xgb[i] >= 0.4)[0])
            comparaciones['productos_xgb'].append(len(np.where(y_pred_xgb[i] >= 0.4)[0]))
            comparaciones['y_pred_nn'].append(np.where(y_pred_nn[i] >= 0.4)[0])
            comparaciones['productos_nn'].append(len(np.where(y_pred_nn[i] >= 0.4)[0]))
        df_comparaciones = pd.DataFrame(comparaciones)
        df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
        df_comparaciones['coincidencias_xgb'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_xgb'])), axis=1)
        df_comparaciones['coincidencias_nn'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_nn'])), axis=1)
        df_comparaciones = df_comparaciones[['usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor', 'y_pred_xgb', 'productos_xgb', 'coincidencias_xgb', 'y_pred_nn', 'productos_nn', 'coincidencias_nn']]
        return df_comparaciones


In [ ]:
def evaluuacion_sin_hiper_ultimo_registro(data:pd.DataFrame, cantidad:int) -> pd.DataFrame:
    df_comparaciones = pd.DataFrame()
    usuarios = [randint(1, 4000) for _ in range(cantidad)]
    for usuario in usuarios:
        try:
            df_1 = modelos_usuario_metricas_ultimo_registro(data, usuario)
            df_comparaciones = pd.concat([df_comparaciones, df_1])
        except:
            continue
    ruta_excel = comparaciones + '/comparaciones_04_ultimo_registro.xlsx'
    df_comparaciones.to_excel(ruta_excel, index=False)
    return df_comparaciones


In [ ]:
df_comparaciones_3 = evaluuacion_sin_hiper_ultimo_registro(df, 50)

1/1 [==============================] - 0s 85ms/step


In [ ]:
metricas(df_comparaciones_3)

Usuario: 3703.0
Precision_regressor: 33.33, Precision_xgb: 0.0, Precision_nn: 16.67
recall_regressor: 57.14, recall_xgb: 0.0, recall_nn: 14.29
f1_score_regressor: 0.42, f1_score_xgb: nan, f1_score_nn: 0.15


Usuario: 2131.0
Precision_regressor: 22.22, Precision_xgb: 0.0, Precision_nn: 42.86
recall_regressor: 44.44, recall_xgb: 0.0, recall_nn: 33.33
f1_score_regressor: 0.3, f1_score_xgb: nan, f1_score_nn: 0.38


Usuario: 2525.0
Precision_regressor: 14.29, Precision_xgb: 0.0, Precision_nn: 0.0
recall_regressor: 16.67, recall_xgb: 0.0, recall_nn: 0.0
f1_score_regressor: 0.15, f1_score_xgb: nan, f1_score_nn: nan


Usuario: 3002.0
Precision_regressor: 66.67, Precision_xgb: nan, Precision_nn: 50.0
recall_regressor: 66.67, recall_xgb: 0.0, recall_nn: 33.33
f1_score_regressor: 0.67, f1_score_xgb: nan, f1_score_nn: 0.4


Usuario: 922.0
Precision_regressor: 33.33, Precision_xgb: nan, Precision_nn: 0.0
recall_regressor: 50.0, recall_xgb: 0.0, recall_nn: 0.0
f1_score_regressor: 0.4, f1_score_xgb: 

##Ajuste de hiperparámetros y prueba de Random Forest

In [ ]:
def hiperparametros_regressor(data:pd.DataFrame, usuario:int, n_estimators:int, max_depth:int, min_samples_split:int, min_samples_leaf:int, criterion:str) -> pd.Dataframe:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    if len(data) == 1:
        return y
    else:
        X_train = X.iloc[:-1,:]
        y_train = y.iloc[:-1,:]
        X_test = X.iloc[-1:,:]
        y_test = y.iloc[-1:,:]
        y_train = y_train.values
        y_test = y_test.values
        X_test = X_test.astype('float32')
        X_train = X_train.astype('float32')
        y_test = y_test.astype('float32')
        y_train = y_train.astype('float32')
        rf_regressor = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, criterion=criterion, random_state=42)
        multi_target_regressor = MultiOutputRegressor(rf_regressor, n_jobs=-1)
        multi_target_regressor.fit(X_train, y_train)
        y_pred_regressor = multi_target_regressor.predict(X_test)
        comparaciones = {'usuario':[],'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[]}
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
        df_comparaciones = pd.DataFrame(comparaciones)
        df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
        df_comparaciones['hiperparametros'] = f'n_estimators={n_estimators}, max_depth={max_depth}, min_samples_split={min_samples_split}, min_samples_leaf={min_samples_leaf}, criterion={criterion}'
        df_comparaciones = df_comparaciones[['hiperparametros', 'usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor']]
        return df_comparaciones


In [ ]:
def evaluacion_rf(data:pd.DataFrame, archivo:str, cantidad:int, n_estimators_l:list, max_depth_l:list, min_samples_split_l:list, min_samples_leaf_l:list, criterion_l:list) -> pd.DataFrame:
    df_comparaciones = pd.DataFrame()
    ruta_excel = comparaciones + archivo
    usuarios = [randint(1, 4000) for _ in range(cantidad)]
    for usuario in usuarios:
        for n_estimators in n_estimators_l:
            for max_depth in max_depth_l:
                for min_samples_split in min_samples_split_l:
                    for min_samples_leaf in min_samples_leaf_l:
                        for criterion in criterion_l:
                            df_1 = hiperparametros_regressor(data, usuario, n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion)
                            df_comparaciones = pd.concat([df_comparaciones, df_1])
                            df_comparaciones.to_excel(ruta_excel, index=False)
    return df_comparaciones


In [ ]:
n_estimators_l = [randint(3,50), randint(3,50), randint(3,50)]
archivo = '/comparaciones_rf.xlsx'
max_depth_l = [3, 4, 5]
min_samples_split_l = [2, 3]
min_samples_leaf_l = [1, 2]
criterion_l = ['squared_error', 'absolute_error']
df_comparaciones_rf = evaluacion_rf(df, archivo, 1, n_estimators_l, max_depth_l, min_samples_split_l, min_samples_leaf_l, criterion_l)

In [ ]:
df_comparaciones_rf

,hiperparametros,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor
0,"n_estimators=19, max_depth=3, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 150]",3,2
0,"n_estimators=19, max_depth=3, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 150]",3,2
0,"n_estimators=19, max_depth=3, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3
0,"n_estimators=19, max_depth=3, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3
0,"n_estimators=19, max_depth=3, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3
...,...,...,...,...,...,...,...
0,"n_estimators=23, max_depth=5, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3
0,"n_estimators=23, max_depth=5, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3
0,"n_estimators=23, max_depth=5, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3
0,"n_estimators=23, max_depth=5, min_samples_spli...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 67, 98, 99, 149, 150, 160, 261, 268, ...",11,3


In [ ]:
n_estimators_l = [20, 50, 70, 100]
archivo = '/comparaciones_rf_2.xlsx'
max_depth_l = [3, 4, 5, 6]
min_samples_split_l = [2, 3]
min_samples_leaf_l = [1, 2, 3]
criterion_l = ['squared_error', 'absolute_error']
df_comparaciones_rf_2 = evaluacion_rf(df, archivo, 2, n_estimators_l, max_depth_l, min_samples_split_l, min_samples_leaf_l, criterion_l)

In [ ]:
df_comparaciones_rf_2

,hiperparametros,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor
0,"n_estimators=20, max_depth=3, min_samples_spli...",1971,"[16, 18, 55, 57, 129, 130, 147, 157, 161]",9,"[16, 26, 55, 57, 94, 137, 147, 156, 157, 161]",10,6
0,"n_estimators=20, max_depth=3, min_samples_spli...",1971,"[16, 18, 55, 57, 129, 130, 147, 157, 161]",9,"[16, 26, 55, 57, 94, 137, 147, 156, 157, 161]",10,6
0,"n_estimators=20, max_depth=3, min_samples_spli...",1971,"[16, 18, 55, 57, 129, 130, 147, 157, 161]",9,"[55, 147, 157]",3,3
0,"n_estimators=20, max_depth=3, min_samples_spli...",1971,"[16, 18, 55, 57, 129, 130, 147, 157, 161]",9,"[34, 48, 55, 129, 147, 157]",6,4
0,"n_estimators=20, max_depth=3, min_samples_spli...",1971,"[16, 18, 55, 57, 129, 130, 147, 157, 161]",9,"[55, 147, 157]",3,3
...,...,...,...,...,...,...,...
0,"n_estimators=100, max_depth=6, min_samples_spl...",1950,"[67, 149, 160, 293]",4,"[14, 18, 44, 46, 48, 56, 62, 103, 109, 116, 12...",21,0
0,"n_estimators=100, max_depth=6, min_samples_spl...",1950,"[67, 149, 160, 293]",4,"[14, 48, 62, 200, 261, 389]",6,0
0,"n_estimators=100, max_depth=6, min_samples_spl...",1950,"[67, 149, 160, 293]",4,"[14, 48, 62, 200, 261, 389]",6,0
0,"n_estimators=100, max_depth=6, min_samples_spl...",1950,"[67, 149, 160, 293]",4,"[14, 62, 149, 261]",4,1


In [ ]:
def hiperparametros_regressor_umbral(data:pd.DataFrame, usuario:int, n_estimators:int, max_depth:int, min_samples_split:int, min_samples_leaf:int, criterion:str, umbral:float) -> pd.Dataframe:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    if len(data) == 1:
        return y
    else:
        X_train = X.iloc[:-1,:]
        y_train = y.iloc[:-1,:]
        X_test = X.iloc[-1:,:]
        y_test = y.iloc[-1:,:]
        y_train = y_train.values
        y_test = y_test.values
        X_test = X_test.astype('float32')
        X_train = X_train.astype('float32')
        y_test = y_test.astype('float32')
        y_train = y_train.astype('float32')
        rf_regressor = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, criterion=criterion, random_state=42)
        multi_target_regressor = MultiOutputRegressor(rf_regressor, n_jobs=-1)
        multi_target_regressor.fit(X_train, y_train)
        y_pred_regressor = multi_target_regressor.predict(X_test)
        comparaciones = {'usuario':[],'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[]}
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= umbral)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= umbral)[0]))
        df_comparaciones = pd.DataFrame(comparaciones)
        df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
        df_comparaciones['hiperparametros'] = f'n_estimators={n_estimators}, max_depth={max_depth}, min_samples_split={min_samples_split}, min_samples_leaf={min_samples_leaf}, criterion={criterion}'
        df_comparaciones = df_comparaciones[['hiperparametros', 'usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor']]
        return df_comparaciones


## Ajuste de hiperparámetros XGBoost

In [ ]:
def hiperparametros_xgb(data:pd.DataFrame, usuario:int, learning_rate:float, max_depth:int, reg_lambda:float, gamma:float, n_estimators:int, objective:str='binary:logistic') -> pd.DataFrame:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    if len(data) == 1:
        return y
    else:
        X_train = X.iloc[:-1,:]
        y_train = y.iloc[:-1,:]
        X_test = X.iloc[-1:,:]
        y_test = y.iloc[-1:,:]
        y_train = y_train.values
        y_test = y_test.values
        X_test = X_test.astype('float32')
        X_train = X_train.astype('float32')
        y_test = y_test.astype('float32')
        y_train = y_train.astype('float')
        xgb_regressor = xgb.XGBRegressor(learning_rate=learning_rate, max_depth=max_depth, reg_lambda=reg_lambda, gamma=gamma, n_estimators=n_estimators, objective=objective, random_state=42)
        multi_target_xgb = MultiOutputRegressor(xgb_regressor, n_jobs=-1)
        multi_target_xgb.fit(X_train, y_train)
        y_pred_regressor = multi_target_xgb.predict(X_test)
        comparaciones = {'usuario':[],'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[]}
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
        df_comparaciones = pd.DataFrame(comparaciones)
        df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
        df_comparaciones['hiperparametros'] = f'learning_rate={learning_rate}, max_depth={max_depth}, reg_lambda={reg_lambda} ,gamma={gamma}, n_estimators={n_estimators}, objective={objective}'
        df_comparaciones = df_comparaciones[['hiperparametros', 'usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor']]
        return df_comparaciones


In [ ]:
def evaluacion_xgb(data:pd.DataFrame, usuario_l:list, archivo:str, learning_rate_l:list, max_depth_l:list, reg_lambda_l:list, gamma_l:list, n_estimators_l:list, objective:list='binary:logistic') -> pd.DataFrame:
    df_comparaciones = pd.DataFrame()
    ruta_excel = comparaciones + archivo
    modelos = 0
    for usuario in usuario_l:
        for learning_rate in learning_rate_l:
            for max_depth in max_depth_l:
                for reg_lambda in reg_lambda_l:
                    for gamma in gamma_l:
                        for n_estimators in n_estimators_l:
                            inicio = time.time()
                            df_1 = hiperparametros_xgb(data, usuario, learning_rate, max_depth, reg_lambda,gamma, n_estimators, objective)
                            df_comparaciones = pd.concat([df_comparaciones, df_1])
                            df_comparaciones.to_excel(ruta_excel, index=False)
                            modelos += 1
                            final = time.time()
                            sys.stdout.write(f'\rModelo número: {modelos}   Tiempo: {round(final-inicio, 2)} segundos')
                            sys.stdout.flush()
    return df_comparaciones


In [ ]:
archivo = '/comparaciones_xgb.xlsx'
usuario_l = [3133, 1971, 1950]
learning_rate_l = [0.1, 0.2, 0.3]
max_depth_l = [3, 4, 5, 7]
gamma_l = [0.1, 0.3]
n_estimators_l = [5, 10, 20, 100]
reg_lambda_l = [0.001, 0.005, 0.2]
inicio = time.time()
df_comparaciones_xgb = evaluacion_xgb(df, usuario_l, archivo, learning_rate_l, max_depth_l, reg_lambda_l, gamma_l, n_estimators_l)
final = time.time()
print(f'Tiempo total: {round(final - inicio, 2)}')

Modelo número: 719   Tiempo: 9.55 segundos

In [ ]:
df_comparaciones_xgb

##Ajuste de hiperparámetros Redes Neuronales

In [ ]:
def hiperparametros_nn(data:pd.DataFrame, usuario:int, activation1:str, activation2:str, activation3:str, learning_rate:float, regularizer:float, optimizer:object, reduce_lr:float, batch_size:int, epochs:int, patience:int) -> pd.DataFrame:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    if len(data) == 1:
        return y
    else:
        X_train = X.iloc[:-1,:]
        y_train = y.iloc[:-1,:]
        X_test = X.iloc[-1:,:]
        y_test = y.iloc[-1:,:]
        y_train = y_train.values
        y_test = y_test.values
        X_test = X_test.astype('float32')
        X_train = X_train.astype('float32')
        y_test = y_test.astype('float32')
        y_train = y_train.astype('float')
        comparaciones = {'usuario':[], 'hiperparametros': [], 'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[]}
        model_1 = Sequential([Dense(256, input_dim=X_train.shape[1], activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(256, activation=activation1),
                            BatchNormalization(),
                            Dense(128, activation=activation1),
                            Dense(64, activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(32, activation=activation1),
                            Dense(y_train.shape[1], activation=activation2)])
        model_1.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics='accuracy')
        model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr, patience=patience, min_lr=1e-8))
        y_pred_regressor = model_1.predict(X_test, verbose=0)
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
            comparaciones['hiperparametros'].append(f'usuario={usuario},  modelo: 1, activation1={activation1}, activation2={activation2}, activation3={activation3}, learning_rate={learning_rate}, regularizer={regularizer}, optimizer={optimizer}, reduce_lr={reduce_lr}, batch_size={batch_size}, epochs={epochs}, patience={patience}')
        model_2 = Sequential([Dense(256, input_dim=X_train.shape[1], activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(256, activation=activation1),
                            BatchNormalization(),
                            Dense(128, activation=activation2),
                            Dense(64, activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(32, activation=activation1),
                            Dense(y_train.shape[1], activation=activation2)])
        model_2.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics='accuracy')
        model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr, patience=patience, min_lr=1e-8))
        y_pred_regressor = model_2.predict(X_test, verbose=0)
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
            comparaciones['hiperparametros'].append(f'usuario={usuario}, modelo: 2, activation1={activation1}, activation2={activation2}, activation3={activation3}, learning_rate={learning_rate}, regularizer={regularizer}, optimizer={optimizer}, reduce_lr={reduce_lr}, batch_size={batch_size}, epochs={epochs}, patience={patience}')
        model_3 = Sequential([Dense(512, input_dim=X_train.shape[1], activation=activation3, kernel_regularizer=l2(regularizer)),
                            Dense(512, activation=activation3),
                            BatchNormalization(),
                            Dense(256, activation=activation3),
                            Dense(128, activation=activation3),
                            BatchNormalization(),
                            Dense(128, activation=activation3),
                            Dense(64, activation=activation3, kernel_regularizer=l2(regularizer)),
                            Dense(32, activation=activation3),
                            Dense(y_train.shape[1], activation=activation2)])
        model_3.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics='accuracy')
        model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr, patience=patience, min_lr=1e-8))
        y_pred_regressor = model_3.predict(X_test, verbose=0)
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.4)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.4)[0]))
            comparaciones['hiperparametros'].append(f'usuario={usuario},  modelo: 3, activation1={activation1}, activation2={activation2}, activation3={activation3}, learning_rate={learning_rate}, regularizer={regularizer}, optimizer={optimizer}, reduce_lr={reduce_lr}, batch_size={batch_size}, epochs={epochs}, patience={patience}')
        df_comparaciones = pd.DataFrame(comparaciones)
        df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
        df_comparaciones = df_comparaciones[['hiperparametros', 'usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor']]
        return df_comparaciones


In [ ]:
def evaluacion_nn(data:pd.DataFrame, usuario_l:list, archivo:str, activation1_:str, activation2_:str, activation3_:str, learning_rate_l:list, regularizer_l:list, optimizer_l:list, reduce_lr_l:list, batch_size_l:list, epochs_l:list, patience_l:list) -> pd.DataFrame:
    df_comparaciones = pd.DataFrame()
    ruta_excel = comparaciones + archivo
    modelos = 0
    for usuario in usuario_l:
        for learning_rate in learning_rate_l:
            for regularizer in regularizer_l:
                for optimizer in optimizer_l:
                    for reduce_lr in reduce_lr_l:
                        for batch_size in batch_size_l:
                            for epochs in epochs_l:
                                for patience in patience_l:
                                    inicio = time.time()
                                    df_1 = hiperparametros_nn(data, usuario, activation1_, activation2_, activation3_, learning_rate, regularizer, optimizer, reduce_lr, batch_size, epochs, patience)
                                    df_comparaciones = pd.concat([df_comparaciones, df_1])
                                    df_comparaciones.to_excel(ruta_excel, index=False)
                                    modelos += 3
                                    final = time.time()
                                    sys.stdout.write(f'\rModelo número: {modelos}   Tiempo: {round(final-inicio, 2)} segundos')
                                    sys.stdout.flush()
    return df_comparaciones


In [ ]:
archivo = '/comparaciones_nn.xlsx'
usuario_l = [3133, 1971, 1950]
learning_rate_l = [0.1, 0.01]
regularizer_l = [0.01]
activation1_ = 'relu'
activation2_ = 'sigmoid'
activation3_ = 'swish'
optimizer_l = [Adam, Nadam]
reduce_lr_l = [0.4]
batch_size_l = [3, 6]
epochs_l = [10, 20]
patience_l = [2, 3]
inicio = time.time()
df_comparaciones_nn = evaluacion_nn(df, usuario_l, archivo, activation1_, activation2_, activation3_, learning_rate_l, regularizer_l, optimizer_l, reduce_lr_l, batch_size_l, epochs_l, patience_l)
final = time.time()
print(f'Tiempo total: {round(final - inicio, 2)}')

Modelo número: 288   Tiempo: 15.55 segundosTiempo total: 1285.71


In [ ]:
df_comparaciones_nn

,hiperparametros,usuario,y_test,productos_test,y_pred_regressor,productos_regressor,coincidencias_regressor
0,"usuario=3133, modelo: 1, activation1=relu, ac...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 150]",3,2
1,"usuario=3133, modelo: 2, activation1=relu, act...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 150]",3,2
2,"usuario=3133, modelo: 3, activation1=relu, ac...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 150]",3,2
0,"usuario=3133, modelo: 1, activation1=relu, ac...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 43, 98, 150]",4,2
1,"usuario=3133, modelo: 2, activation1=relu, act...",3133,"[2, 5, 18, 149, 150, 199, 208]",7,"[18, 67, 98, 99, 149, 150, 160, 261, 371]",9,3
...,...,...,...,...,...,...,...
1,"usuario=1950, modelo: 2, activation1=relu, act...",1950,"[67, 149, 160, 293]",4,"[14, 62, 149, 261]",4,1
2,"usuario=1950, modelo: 3, activation1=relu, ac...",1950,"[67, 149, 160, 293]",4,"[14, 44, 46, 48, 56, 62, 103, 116, 129, 158, 1...",17,0
0,"usuario=1950, modelo: 1, activation1=relu, ac...",1950,"[67, 149, 160, 293]",4,"[14, 44, 48, 62, 149, 150, 159, 200, 261, 262,...",11,1
1,"usuario=1950, modelo: 2, activation1=relu, act...",1950,"[67, 149, 160, 293]",4,"[14, 62, 149, 261]",4,1


In [ ]:
def hiperparametros_nn_05(data:pd.DataFrame, usuario:int, activation1:str, activation2:str, activation3:str, learning_rate:float, regularizer:float, optimizer:object, reduce_lr:float, batch_size:int, epochs:int, patience:int) -> pd.DataFrame:
    data = data[data['id_usuario']==usuario]
    features = ['perfil_digital', 'mes', 'dia', 'dia_semana', 'nse', 'segmento_unico']
    scaler = MinMaxScaler()
    data[features] = scaler.fit_transform(data[features])
    features_total = features + list(data.filter(regex='^canal_').columns)
    target = data.filter(regex='^producto_').columns
    X = data[features_total]
    y = data[target]
    if len(data) == 1:
        return y
    else:
        X_train = X.iloc[:-1,:]
        y_train = y.iloc[:-1,:]
        X_test = X.iloc[-1:,:]
        y_test = y.iloc[-1:,:]
        y_train = y_train.values
        y_test = y_test.values
        X_test = X_test.astype('float32')
        X_train = X_train.astype('float32')
        y_test = y_test.astype('float32')
        y_train = y_train.astype('float')
        comparaciones = {'usuario':[], 'hiperparametros': [], 'y_test':[], 'productos_test':[], 'y_pred_regressor':[], 'productos_regressor':[]}
        model_1 = Sequential([Dense(256, input_dim=X_train.shape[1], activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(256, activation=activation1),
                            BatchNormalization(),
                            Dense(128, activation=activation1),
                            Dense(64, activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(32, activation=activation1),
                            Dense(y_train.shape[1], activation=activation2)])
        model_1.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics='accuracy')
        model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr, patience=patience, min_lr=1e-8))
        y_pred_regressor = model_1.predict(X_test, verbose=0)
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.5)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.5)[0]))
            comparaciones['hiperparametros'].append(f'usuario={usuario},  modelo: 1, activation1={activation1}, activation2={activation2}, activation3={activation3}, learning_rate={learning_rate}, regularizer={regularizer}, optimizer={optimizer}, reduce_lr={reduce_lr}, batch_size={batch_size}, epochs={epochs}')
        model_2 = Sequential([Dense(256, input_dim=X_train.shape[1], activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(256, activation=activation1),
                            BatchNormalization(),
                            Dense(128, activation=activation2),
                            Dense(64, activation=activation1, kernel_regularizer=l2(regularizer)),
                            Dense(32, activation=activation1),
                            Dense(y_train.shape[1], activation=activation2)])
        model_2.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics='accuracy')
        model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr, patience=patience, min_lr=1e-8))
        y_pred_regressor = model_2.predict(X_test, verbose=0)
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.5)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.5)[0]))
            comparaciones['hiperparametros'].append(f'usuario={usuario}, modelo: 2, activation1={activation1}, activation2={activation2}, activation3={activation3}, learning_rate={learning_rate}, regularizer={regularizer}, optimizer={optimizer}, reduce_lr={reduce_lr}, batch_size={batch_size}, epochs={epochs}')
        model_3 = Sequential([Dense(512, input_dim=X_train.shape[1], activation=activation3, kernel_regularizer=l2(regularizer)),
                            Dense(512, activation=activation3),
                            BatchNormalization(),
                            Dense(256, activation=activation3),
                            Dense(128, activation=activation3),
                            BatchNormalization(),
                            Dense(128, activation=activation3),
                            Dense(64, activation=activation3, kernel_regularizer=l2(regularizer)),
                            Dense(32, activation=activation3),
                            Dense(y_train.shape[1], activation=activation2)])
        model_3.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics='accuracy')
        model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr, patience=patience, min_lr=1e-8))
        y_pred_regressor = model_3.predict(X_test, verbose=0)
        for i in range(len(y_test)):
            comparaciones['usuario'].append(usuario)
            comparaciones['y_test'].append(np.where(y_test[i] ==1)[0])
            comparaciones['productos_test'].append(len(np.where(y_test[i] ==1)[0]))
            comparaciones['y_pred_regressor'].append(np.where(y_pred_regressor[i] >= 0.5)[0])
            comparaciones['productos_regressor'].append(len(np.where(y_pred_regressor[i] >= 0.5)[0]))
            comparaciones['hiperparametros'].append(f'usuario={usuario},  modelo: 3, activation1={activation1}, activation2={activation2}, activation3={activation3}, learning_rate={learning_rate}, regularizer={regularizer}, optimizer={optimizer}, reduce_lr={reduce_lr}, batch_size={batch_size}, epochs={epochs}')
        df_comparaciones = pd.DataFrame(comparaciones)
        df_comparaciones['coincidencias_regressor'] = df_comparaciones.apply(lambda row: len(set(row['y_test']) & set(row['y_pred_regressor'])), axis=1)
        df_comparaciones = df_comparaciones[['hiperparametros', 'usuario', 'y_test', 'productos_test', 'y_pred_regressor', 'productos_regressor', 'coincidencias_regressor']]
        return df_comparaciones


In [ ]:
def evaluacion_nn_05(data:pd.DataFrame, usuario_l:list, archivo:str, activation1_:str, activation2_:str, activation3_:str, learning_rate_l:list, regularizer_l:list, optimizer_l:list, reduce_lr_l:list, batch_size_l:list, epochs_l:list, patience_l:list) -> pd.DataFrame:
    df_comparaciones = pd.DataFrame()
    ruta_excel = comparaciones + archivo
    modelos = 0
    for usuario in usuario_l:
        for learning_rate in learning_rate_l:
            for regularizer in regularizer_l:
                for optimizer in optimizer_l:
                    for reduce_lr in reduce_lr_l:
                        for batch_size in batch_size_l:
                            for epochs in epochs_l:
                                for patience in patience_l:
                                    inicio = time.time()
                                    df_1 = hiperparametros_nn_05(data, usuario, activation1_, activation2_, activation3_, learning_rate, regularizer, optimizer, reduce_lr, batch_size, epochs, patience)
                                    df_comparaciones = pd.concat([df_comparaciones, df_1])
                                    df_comparaciones.to_excel(ruta_excel, index=False)
                                    modelos += 3
                                    final = time.time()
                                    sys.stdout.write(f'\rModelo número: {modelos}   Tiempo: {round(final-inicio, 2)} segundos')
                                    sys.stdout.flush()
    return df_comparaciones


In [ ]:
archivo = '/comparaciones_nn_sgd.xlsx'
usuario_l = [3133, 1971, 1950]
learning_rate_l = [0.1, 0.01]
regularizer_l = [0.001, 0.1, 1]
activation1_ = 'relu'
activation2_ = 'sigmoid'
activation3_ = 'swish'
optimizer_l = [SGD]
reduce_lr_l = [0.99999, 0.2]
batch_size_l = [1, 3, 6]
epochs_l = [10, 20, 30]
patience_l = [2, 3]
inicio = time.time()
df_comparaciones_nn_sgd = evaluacion_nn_05(df, usuario_l, archivo, activation1_, activation2_, activation3_, learning_rate_l, regularizer_l, optimizer_l, reduce_lr_l, batch_size_l, epochs_l, patience_l)
final = time.time()
print(f'Tiempo total: {round(final - inicio, 2)}')

Modelo número: 3   Tiempo: 12.2 segundos

Modelo número: 156   Tiempo: 9.97 segundos

KeyboardInterrupt: 

In [ ]:
df_comparaciones_nn_sgd